<a href="https://colab.research.google.com/github/thiensean/Grounded-Segment-Anything/blob/main/groundedSAM_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
import numpy as np
import torch
from PIL import Image
from io import BytesIO

app = FastAPI()

class GenerateRequest(BaseModel):
    mask_id: int
    prompt: str

@app.post("/upload")
async def upload_image(file: UploadFile = File(...)):
    # Read the uploaded image
    image_data = await file.read()
    image = Image.open(BytesIO(image_data))

    # Process the image to detect and segment objects, generating masks
    masks = process_image(image)

    # Return mask IDs to the user
    return {"mask_ids": list(range(len(masks)))}

@app.post("/generate")
def generate_image(request: GenerateRequest):
    # Use the mask ID and prompt to generate an image
    generated_image = create_image_with_prompt(request.mask_id, request.prompt)

    # Convert the generated image to a format that can be sent to the user
    img_byte_arr = BytesIO()
    generated_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    return Response(content=img_byte_arr, media_type="image/png")
